<h1><center>Walmart</center></h1>
<h2><center>Two Index Vehicle Routing Formulation</center></h2>

Developed by: Daniel A. Zuniga Vazquez\
Date: 10/02/2021\
Version 1.0.0 

Sets and indices\
$D$: Set of destinations, indexed by $i$, $j$.

Parameters\
$c_{ij}$: Vehicle routing cost from vertex $i$ to vertex $j$.\
$K$: Number of vehicles.\
$MAX$: Maximum number of destinations a vehicle can be routed to.

Variables\
$x_{ij}$: Binary variables that is 1 if a vehicle is routed from destination $i$ to destination $j$ and 0 otherwise.\
$y_i$: Integer variable that denotes the destination $i$ position in the vehicle routing.

Problem Formulation\
$
\begin{align}
\min_{\pmb{x},\pmb{y}}~& \sum_{i \in D} \sum_{j \in D} c_{ij} x_{ij} & \text{Minimize total cost}\\
    \text{s.t. } &  \sum_{i \in D} x_{ij} = 1, \forall j \in D \setminus \{0\}: j \neq i  & \text{1.1a - Only one vehicle can enter a destination}\\
    & \sum_{j \in D} x_{ij} = 1, \forall i \in D \setminus \{0\}: i \neq j & \text{1.1b - Only one vehicle can leave a destination}\\
    & \sum_{i \in D \setminus \{0\}} x_{i0} = K & \text{1.1c - Number of vehicles leaving the depot at destination 0}\\
    & \sum_{j \in D \setminus \{0\}} x_{0j} = K & \text{1.1d - Number of vehicles entering the depot at destination 0}\\
    & y_i - y_j + MAX~x_{ij} \leq MAX -1,~\forall i, j \in D \setminus \{0\} : i \neq j  & \text{1.1e - Subtour elimination (Miller-Tucker-Zemlin)}\\
    & x_{i,j} \in \{0,1\}, y_i \in \mathbb{Z},~\forall i \in I, \forall j \in J & \text{1.1f - Domain}
\end{align}
$


In [ ]:
# For purposes of this Examples tutorial, we will restart the kernel for each problem
import os
os._exit(00)

In [1]:
# To plot interactable figures and print in notebook
%matplotlib notebook

# Libraries
import pandas as pd                #data manipulation and analysis library
import numpy as np                 #array and matrices library
from docplex.mp.model import Model #cplex library

from IPython.core.debugger import set_trace #Library to help with debgging      Use set_trace()   if breakpoint is needed
# Debugging commands ONLY use first letter
# n(ext) execute the current statement (step over)
# s(tep) execute and step into function
# r(eturn) continue execution until the current function returns
# c(ontinue) continue execution until a breakpoint is encountered
# u(p) move one level up in the stack trace
# d(own) move one level down in the stack trace

In [2]:
# Reading parameters from external TXT
#*******************************************
# NOTE: for this purpose, a dummy initial row of zeros is added in the txt file matching the length of the last column

# File name must be in the same directory as the python file, or you can use the complete path if not
fileName = 'Walmart_TwoIndexVehicleRouting.txt'

# Identify the length of the column
columns = max(len(l.split()) for l in open(fileName))

# Use panda read_table function to read all the values in a table
tableValues = pd.read_table(fileName, 
                    delim_whitespace=True, 
                    header=None, 
                    usecols=range(columns), 
                    engine='python')

# Transform the table values into a numpy array, missing values will be identified as NaN just to complete the matrix
# Note that the dummy initial row of zeros is still there and integers will be read as float, so the integer conversion
# required
iData = np.array(tableValues)

# Declare parameters
D = int(iData[1][0])       # Set of destinations, cardinality

# c[i][j]: Vehicle routing cost from vertex i to vertex j. 
c = {(i,j):int((iData[i+2][j])) for i in range(D) for j in range(D)} 

K = int(iData[D+3][0])     # Number of vehicles
MAX = int(iData[D+3][0])   # Maximum number of destinations a vehicle can be routed to


In [3]:
# Creates cplex model
mycplex = Model('TwoIndexVehicleRouting_PYTHON')   #Name of Model

In [4]:
# Declare variables

# Auxiliary array to create binary variable name
nameX = [(i,j) for i in range(D) for j in range(D)] 
# x[i][j]: Binary variables that is 1 if a vehicle is routed from destination i to destination j and 0 otherwise.
x = mycplex.binary_var_dict(nameX, name='x')

# Auxiliary array to create integer variable name
nameY = [(i) for i in range(D)]  
# y[i]: Integer variable that denotes the destination i position in the vehicle routing.
y = mycplex.integer_var_dict(nameY, lb=0, ub=None, name='y',)


In [5]:
# Create objective

OBJ = 0
for i in range(D):
    for j in range(D):
        OBJ += c[i,j] * x[i,j]

mycplex.minimize(OBJ) 

In [6]:
# Adding constraints

# Constraint 1.1a - Only one vehicle can enter a destination
for j in range(1,D):
    Cons1a = 0
    for i in range(D):
        if(i!=j):
            Cons1a += x[i,j]
    mycplex.add_constraint(Cons1a == 1)

# Constraint 1.1b - Only one vehicle can leave a destination
for i in range(1,D):
    Cons1b = 0
    for j in range(D):
            if(i!=j):
                Cons1b += x[i,j]
    mycplex.add_constraint(Cons1b == 1)
    
# Constraint 1.1c - Number of vehicles leaving the depot at destination 0
mycplex.add_constraint(sum(x[i,0] for i in range(1,D)) == K)

# Constraint 1.1d - Number of vehicles entering the depot at destination 0
mycplex.add_constraint(sum(x[0,j] for j in range(1,D)) == K)

# Constraint 1.1e - Subtour elimination (Miller-Tucker-Zemlin)   
for j in range(1,D):
    for i in range(D):
        if(i!=j):
            mycplex.add_constraint(y[i] - y[j] + (MAX * x[i,j]) <= MAX - 1)

In [7]:
#Export cplex model
mycplex.export_as_lp('TwoIndexVehicleRouting_PYTHON')  # exports model in lp format
mycplex.export_as_mps('TwoIndexVehicleRouting_PYTHON') # exports model in mps format

'TwoIndexVehicleRouting_PYTHON.mps'

In [8]:
# Solve Cplex model
solution = mycplex.solve(log_output = True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 18 columns.
MIP Presolve modified 32 coefficients.
Reduced MIP has 290 rows, 288 columns, and 1296 nonzeros.
Reduced MIP has 272 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.70 ticks)
Probing time = 0.02 sec. (0.75 ticks)
Cover probing fixed 0 vars, tightened 16 bounds.
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 16 rows and 0 columns.
Reduced MIP has 274 rows, 288 columns, and 1264 nonzeros.
Reduced MIP has 272 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.90 ticks)
Probing time = 0.02 sec. (0.74 ticks)
Clique table members: 152.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.46 ticks)

        No

In [9]:
# Print Solutions
print(solution)

solution for: TwoIndexVehicleRouting_PYTHON
objective: 7704
x_0_5=1
x_0_7=1
x_0_8=1
x_0_9=1
x_0_12=1
x_0_13=1
x_1_0=1
x_2_10=1
x_3_4=1
x_4_1=1
x_5_0=1
x_6_2=1
x_7_0=1
x_8_6=1
x_9_0=1
x_10_16=1
x_11_15=1
x_12_11=1
x_13_0=1
x_14_0=1
x_15_3=1
x_16_14=1
y_1=6
y_2=3
y_3=4
y_4=5
y_5=1
y_6=2
y_7=1
y_8=1
y_9=1
y_10=4
y_11=2
y_12=1
y_13=1
y_14=6
y_15=3
y_16=5

